In [ ]:
include("./KeplerSolver.jl")
include("./Doppler.jl")
using Doppler
using KeplerSolver
using PyPlot
using DataFrames
using LsqFit

In [ ]:
# get the radial velocity data

rv_data = readtable("mystery_planet1.txt", separator = ' ');
t = rv_data[1];
v = rv_data[2];
err = rv_data[3];
t = t - t[1];

In [ ]:
# functions to fold radial velocity curve and to compute the sum of the differences between adjacent RV points

function fold(t, t0)
    t = t .% t0 
end

function adjacent_diffs(v)
    sum = 0
    for i in 1:length(v)
        j = i + 1
        if j > length(v)
            j = 1
        end
        sum = sum + abs(v[i] - v[j])
    end
    return sum
end;

In [ ]:
# find the correct period to fold on

min = adjacent_diffs(v)
t0_min = 0
t_min = t[1]
t_max = t[length(t)]
for t0 in linspace(t_min, t_max, 10000)
    p = sortperm(fold(t, t0))
    diffs = adjacent_diffs(v[p])
    if diffs < min
        min = diffs
        t0_min = t0
    end
end 
folded_t = fold(t, t0_min);

In [ ]:
# model of RV curve

function model(t, params)
    P, ecc, tp, gamma, omega, K = params
    return vrad(t, P, ecc, tp, gamma, omega, K)
end;

In [ ]:
# used this cell to guess initial parameters for LsqFit
 plot(folded_t, v, ".") 
 plot(folded_t, model(folded_t, [100,0.9,75,0,180, 0.5]), ".")

In [ ]:
# fit using LsqFit

fit = curve_fit(model, convert(Array, folded_t), convert(Array, v), convert(Array, err), [100,0.9,75,0,180, 0.5]);

In [ ]:
# plot the fit and print the parameters

plot(folded_t, v, ".")
plot(folded_t, model(folded_t, fit.param), ".")
println("params [P, ecc, tp, gamma, omega, K] = ", fit.param)